# Diffraction Limit Simulation

Simulating the output of a prime focus telescope given an input image. 

## Kitt Peak 2.1m Telescope
From http://www.noao.edu/outreach/kptour/2_1_m.html, this telescope has the following specs:

- Diameter = 2.1m = 2100mm
- Focal Ratio = f/2.63
- Focal Length = 2100mm*2.63 = 5523mm

## Andora Luca-R
From https://www.andor.com/pdfs/specifications/Andor_Luca-R_604_Specifications.pdf

- Sensor Size = 1004 (H) x 1002 (V) Pixels = 8mm X 8mm
 - Simplifying to 1000 x 1000 Pixels
- Pixel Size = 8um X 8um

## Calcs:
- Input image is in units of angles [ex: arcsec], represents the full input field of view
- Aperture is in units of wavelength [ex: um]
- Output image is in units of pixels

In [2]:
# Includes
import matplotlib.pyplot as plt
import numpy as np
from scipy.fftpack import fft2, ifft2

%matplotlib inline
# Defining Convolution Function

# Creating Input Image

# Creating Aperture Image

# Calculate Output Image